# Learning Functional ANOVA Decompositions in FAVA: Continous Responses

In this tutorial, we show how to solve penalized least squares problems of the form

$$\hat{f} = \arg \min_{f \in H} \sum_{n=1}^N (y^{(n)} - f(x^{(n)}))^2 + \lambda \|f\|_{\mathcal{H}}^2, $$

where $\mathcal{H}$ consists of the space of interactions of order up to $Q$: 

$$\mathcal{H} = \left\{f: f = \sum_{V: |V| \leq Q} f_V \right\} \quad \text{s.t.} \quad \forall A \subsetneq B, \ f_A \perp_{\mathcal{H}} f_B.$$

# Import packages

We start by importing the required packages in `fava` and `jax`.

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import jax

from fava.inference.fit import GaussianSKIMFA
from fava.basis.maps import LinearBasis
from fava.misc.scheduler import truncScheduler
from fava.misc.logger import GausLogger
from fava.decomposers.decomposer import all_subsets
from fava.decomposers.tensor_product import TensorProductKernelANOVA, LinearANOVA

# Simulating data

Below we generate simulated data, where the covariates $x \in \mathbb{R}^{200}$ and $y \in \mathbb{R}$. We assume that the first 4 covariates drive the response $y$, and the remaining covariates have no influence on the response. Specifically, the $n$th datapoint is generated as follows: $y^{(n)} = x_1^{(n)}x_2^{(n)} + x_2^{(n)}x_3^{(n)} + \epsilon^{(n)}$, where $\epsilon^{(n)} \overset{\text{i.i.d.}}{\sim} \mathcal{N}(0, 1)$ for $1 \leq n \leq 500$.


In [2]:
key = random.PRNGKey(0) # set seed
N = 500
p = 100
X = random.normal(key, shape=(N, p))
epsilon = random.normal(key, shape=(N, ))
Y = X[:, 0] + X[:, 1] + X[:, 2] * X[:, 3] + epsilon

# Split data into training and validation

In [3]:
# Use 80% of the data for training and 20% for validation
X_train = X[:400, :]
Y_train = Y[:400]

X_valid = X[400:, :]
Y_valid = Y[400:]

# Fit SKIM-FA Model

Below we initialize SKIM-FA hyperparameters with default values.

In [4]:
kernel_params = dict()
Q = 2
kernel_params['U_tilde'] = jnp.ones(p)
kernel_params['eta'] = jnp.ones(Q+1)

hyperparams = dict()
hyperparams['sigma_sq'] = .5 * jnp.var(Y)
hyperparams['c'] = .1

opt_params = dict()
opt_params['cg'] = True
opt_params['cg_tol'] = .01
opt_params['M'] = 50
opt_params['gamma'] = .1
opt_params['T'] = 1000
opt_params['scheduler'] = truncScheduler()

featprocessor = LinearBasis(X_train)
logger = GausLogger(100)

skim = GaussianSKIMFA(X_train, Y_train, X_valid, Y_valid, featprocessor)

skim.fit(key, hyperparams, kernel_params, opt_params, 
            logger=GausLogger())

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]

============================== Iteration 0/1000 ==============================
There are 100 covariates selected.


  0%|                                                                                                      | 1/1000 [00:04<1:10:48,  4.25s/it]

MSE (Validation)=3.0428.
R2 (Validation)=0.3288.
eta=[0.9990643 1.0990986 0.9013535]
c=0.0
kappa=[0.53514034 0.5359756  0.5041889  0.5089301  0.4982728  0.4977355
 0.5020504  0.50118136 0.4985874  0.50389117 0.5028446  0.4957614
 0.5015027  0.50450885 0.4992278  0.49951398 0.49762988 0.50137377
 0.500564   0.49599436 0.4923414  0.50306463 0.49447912 0.50221264
 0.49255246 0.50433195 0.49720705 0.5052371  0.49460822 0.4971692
 0.49368238 0.5075953  0.49819574 0.49350464 0.50884753 0.48530623
 0.49458608 0.4947715  0.5056628  0.49032962 0.5049976  0.49886423
 0.49705204 0.50485826 0.4941874  0.50526375 0.49185562 0.5042454
 0.50271714 0.49430764 0.498645   0.49878007 0.5008558  0.5016438
 0.48900688 0.49597877 0.49731752 0.49453175 0.48636627 0.49595442
 0.50288916 0.50344175 0.49673682 0.5066955  0.49874645 0.49971017
 0.49290466 0.49842602 0.50236243 0.49415013 0.50040877 0.49763352
 0.49582794 0.49856067 0.50668097 0.4965555  0.49432254 0.49839714
 0.49864483 0.5046699  0.49489406 0.5

 10%|██████████▏                                                                                           | 100/1000 [00:47<06:35,  2.28it/s]

...

 90%|███████████████████████████████████████████████████████████████████████████████████████████▊          | 900/1000 [06:27<00:38,  2.62it/s]

============================== Iteration 900/1000 ==============================
There are 56 covariates selected.


 90%|███████████████████████████████████████████████████████████████████████████████████████████▉          | 901/1000 [06:28<00:49,  1.98it/s]

MSE (Validation)=2.9698.
R2 (Validation)=0.3449.
eta=[0.9313462 1.0909448 1.7465771]
c=0.1721
kappa=[0.7319947  0.5512021  0.4466469  0.48950973 0.         0.
 0.4666628  0.58701587 0.         0.22745213 0.         0.18970713
 0.30220318 0.5042257  0.         0.29979506 0.         0.35709915
 0.31215456 0.09318838 0.         0.         0.20504114 0.
 0.07211573 0.43019137 0.         0.24004701 0.         0.
 0.2992889  0.45716035 0.35606545 0.09021855 0.7402805  0.
 0.         0.         0.3674835  0.27910113 0.23930395 0.19555204
 0.         0.38201421 0.         0.6283059  0.         0.42819747
 0.49102724 0.         0.         0.         0.649228   0.69699717
 0.         0.         0.76433235 0.2784943  0.1140749  0.798218
 0.7192539  0.40033832 0.         0.6967953  0.         0.28464258
 0.         0.05080554 0.         0.         0.2923443  0.35271776
 0.28743446 0.         0.40118486 0.         0.14083299 0.
 0.         0.74947035 0.         0.3357854  0.5204915  0.
 0.         

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:05<00:00,  2.35it/s]


# Report nonzero interaction effects

Now we compute all of the non-zero interaction effects found by SKIM-FA. Since we fit a linear interaction model, we report the regression coefficients associated with each effect.

In [12]:
lanova = LinearANOVA(skim)
V_all = all_subsets(skim.selected_covariates(), 2, True)
for V in V_all:
    print(V, round(lanova.get_coef(V), 3))

() 0.018
(0,) 0.747
(1,) 0.8
(2,) -0.117
(3,) 0.017
(4,) -0.016
(5,) -0.02
(6,) -0.042
(7,) 0.013
(8,) 0.026
(9,) -0.055
(10,) -0.096
(11,) -0.014
(12,) -0.074
(13,) -0.05
(14,) -0.143
(15,) -0.006
(16,) -0.046
(17,) -0.124
(18,) -0.075
(19,) 0.08
(20,) 0.081
(21,) -0.051
(22,) 0.071
(23,) 0.08
(24,) -0.079
(25,) -0.053
(26,) -0.001
(27,) -0.071
(28,) -0.003
(29,) -0.064
(30,) 0.127
(31,) -0.054
(32,) -0.016
(33,) 0.016
(34,) -0.012
(35,) -0.012
(36,) -0.016
(37,) -0.018
(38,) -0.035
(39,) -0.108
(40,) -0.06
(41,) 0.028
(42,) -0.027
(43,) 0.063
(44,) 0.05
(45,) 0.028
(46,) 0.015
(47,) 0.004
(48,) 0.034
(49,) -0.012
(50,) -0.072
(51,) 0.001
(52,) -0.098
(53,) -0.069
(54,) -0.002
(55,) 0.024
(56,) 0.014
(57,) 0.011
(58,) -0.004
(59,) -0.073
(60,) 0.057
(61,) 0.034
(62,) 0.023
(63,) -0.003
(64,) -0.074
(65,) 0.1
(66,) 0.031
(67,) 0.023
(68,) -0.071
(69,) -0.035
(70,) -0.111
(71,) -0.006
(72,) -0.026
(73,) -0.049
(74,) -0.038
(75,) 0.017
(76,) 0.019
(77,) -0.056
(78,) -0.022
(79,) -0.002
(

(5, 40) 0.016
(5, 41) -0.007
(5, 42) 0.0
(5, 43) 0.007
(5, 44) -0.001
(5, 45) -0.02
(5, 46) -0.001
(5, 47) -0.025
(5, 48) -0.006
(5, 49) -0.002
(5, 50) 0.013
(5, 51) -0.02
(5, 52) 0.033
(5, 53) -0.021
(5, 54) 0.001
(5, 55) -0.002
(5, 56) 0.018
(5, 57) -0.004
(5, 58) -0.003
(5, 59) 0.001
(5, 60) 0.02
(5, 61) -0.004
(5, 62) -0.002
(5, 63) 0.029
(5, 64) -0.002
(5, 65) 0.016
(5, 66) -0.004
(5, 67) -0.0
(5, 68) 0.009
(5, 69) 0.002
(5, 70) 0.004
(5, 71) -0.014
(5, 72) -0.002
(5, 73) 0.003
(5, 74) 0.015
(5, 75) 0.002
(5, 76) 0.001
(5, 77) -0.003
(5, 78) 0.004
(5, 79) -0.028
(5, 80) -0.005
(5, 81) -0.024
(5, 82) 0.018
(5, 83) -0.001
(5, 84) -0.0
(5, 85) 0.011
(5, 86) 0.008
(5, 87) -0.004
(5, 88) 0.003
(5, 89) 0.012
(5, 90) -0.008
(5, 91) 0.004
(5, 92) -0.001
(5, 93) 0.002
(5, 94) -0.003
(5, 95) -0.003
(5, 96) -0.002
(5, 97) 0.005
(5, 98) -0.0
(5, 99) 0.009
(6, 7) -0.032
(6, 8) -0.065
(6, 9) -0.035
(6, 10) 0.015
(6, 11) -0.032
(6, 12) 0.04
(6, 13) 0.029
(6, 14) -0.01
(6, 15) -0.014
(6, 16) -0.0

(11, 74) 0.087
(11, 75) 0.012
(11, 76) 0.029
(11, 77) -0.001
(11, 78) 0.014
(11, 79) 0.015
(11, 80) 0.025
(11, 81) 0.042
(11, 82) -0.04
(11, 83) -0.002
(11, 84) 0.006
(11, 85) 0.026
(11, 86) 0.004
(11, 87) 0.019
(11, 88) 0.006
(11, 89) -0.029
(11, 90) 0.008
(11, 91) -0.022
(11, 92) 0.009
(11, 93) -0.054
(11, 94) -0.008
(11, 95) -0.041
(11, 96) -0.0
(11, 97) -0.025
(11, 98) -0.006
(11, 99) 0.035
(12, 13) -0.056
(12, 14) 0.004
(12, 15) -0.051
(12, 16) -0.003
(12, 17) 0.004
(12, 18) -0.031
(12, 19) -0.08
(12, 20) -0.015
(12, 21) -0.005
(12, 22) -0.042
(12, 23) -0.004
(12, 24) -0.014
(12, 25) -0.023
(12, 26) -0.001
(12, 27) -0.056
(12, 28) 0.003
(12, 29) -0.005
(12, 30) 0.008
(12, 31) -0.003
(12, 32) -0.04
(12, 33) -0.011
(12, 34) -0.021
(12, 35) 0.001
(12, 36) 0.029
(12, 37) 0.017
(12, 38) -0.077
(12, 39) 0.005
(12, 40) -0.018
(12, 41) 0.003
(12, 42) -0.016
(12, 43) -0.029
(12, 44) -0.0
(12, 45) 0.02
(12, 46) -0.001
(12, 47) -0.037
(12, 48) 0.062
(12, 49) 0.011
(12, 50) 0.021
(12, 51) 0.0

(18, 52) 0.07
(18, 53) 0.033
(18, 54) 0.002
(18, 55) -0.001
(18, 56) -0.059
(18, 57) -0.022
(18, 58) 0.01
(18, 59) 0.033
(18, 60) 0.039
(18, 61) 0.086
(18, 62) -0.049
(18, 63) 0.014
(18, 64) 0.027
(18, 65) 0.022
(18, 66) 0.018
(18, 67) -0.024
(18, 68) 0.002
(18, 69) -0.006
(18, 70) -0.01
(18, 71) 0.002
(18, 72) 0.03
(18, 73) 0.013
(18, 74) -0.048
(18, 75) -0.005
(18, 76) -0.052
(18, 77) -0.008
(18, 78) -0.001
(18, 79) -0.029
(18, 80) -0.006
(18, 81) -0.062
(18, 82) 0.014
(18, 83) 0.005
(18, 84) 0.003
(18, 85) 0.076
(18, 86) 0.003
(18, 87) 0.089
(18, 88) 0.01
(18, 89) 0.021
(18, 90) 0.043
(18, 91) -0.049
(18, 92) -0.056
(18, 93) 0.033
(18, 94) 0.01
(18, 95) -0.018
(18, 96) -0.005
(18, 97) -0.061
(18, 98) -0.03
(18, 99) -0.039
(19, 20) -0.034
(19, 21) 0.046
(19, 22) -0.033
(19, 23) 0.022
(19, 24) -0.012
(19, 25) 0.07
(19, 26) 0.024
(19, 27) -0.036
(19, 28) -0.004
(19, 29) 0.021
(19, 30) 0.006
(19, 31) -0.063
(19, 32) 0.06
(19, 33) -0.056
(19, 34) -0.003
(19, 35) -0.003
(19, 36) -0.002
(1

(25, 48) -0.031
(25, 49) 0.003
(25, 50) -0.015
(25, 51) 0.051
(25, 52) 0.04
(25, 53) 0.093
(25, 54) -0.012
(25, 55) 0.003
(25, 56) -0.107
(25, 57) -0.004
(25, 58) 0.014
(25, 59) -0.002
(25, 60) -0.125
(25, 61) -0.041
(25, 62) -0.001
(25, 63) -0.134
(25, 64) -0.045
(25, 65) 0.034
(25, 66) -0.0
(25, 67) -0.008
(25, 68) -0.022
(25, 69) 0.007
(25, 70) 0.01
(25, 71) -0.006
(25, 72) 0.057
(25, 73) 0.004
(25, 74) -0.223
(25, 75) -0.006
(25, 76) -0.002
(25, 77) 0.008
(25, 78) -0.037
(25, 79) -0.026
(25, 80) 0.011
(25, 81) -0.045
(25, 82) 0.009
(25, 83) 0.017
(25, 84) -0.007
(25, 85) 0.033
(25, 86) 0.027
(25, 87) 0.003
(25, 88) 0.036
(25, 89) 0.002
(25, 90) 0.043
(25, 91) -0.004
(25, 92) 0.002
(25, 93) -0.075
(25, 94) 0.032
(25, 95) -0.039
(25, 96) 0.004
(25, 97) -0.014
(25, 98) 0.049
(25, 99) 0.005
(26, 27) 0.047
(26, 28) 0.018
(26, 29) -0.039
(26, 30) 0.015
(26, 31) -0.027
(26, 32) 0.003
(26, 33) -0.007
(26, 34) -0.033
(26, 35) 0.003
(26, 36) 0.016
(26, 37) 0.006
(26, 38) 0.01
(26, 39) 0.012


(33, 63) 0.044
(33, 64) -0.037
(33, 65) -0.014
(33, 66) -0.017
(33, 67) -0.015
(33, 68) -0.031
(33, 69) 0.002
(33, 70) 0.048
(33, 71) -0.004
(33, 72) 0.007
(33, 73) 0.005
(33, 74) 0.017
(33, 75) -0.007
(33, 76) -0.045
(33, 77) -0.005
(33, 78) -0.031
(33, 79) -0.01
(33, 80) 0.006
(33, 81) 0.069
(33, 82) -0.053
(33, 83) -0.011
(33, 84) -0.03
(33, 85) -0.017
(33, 86) -0.008
(33, 87) 0.016
(33, 88) 0.053
(33, 89) 0.005
(33, 90) -0.03
(33, 91) 0.011
(33, 92) 0.02
(33, 93) -0.005
(33, 94) 0.002
(33, 95) -0.036
(33, 96) 0.004
(33, 97) -0.007
(33, 98) -0.028
(33, 99) 0.012
(34, 35) 0.0
(34, 36) -0.019
(34, 37) -0.082
(34, 38) 0.064
(34, 39) -0.033
(34, 40) 0.011
(34, 41) -0.046
(34, 42) -0.012
(34, 43) 0.094
(34, 44) 0.001
(34, 45) -0.013
(34, 46) -0.011
(34, 47) 0.058
(34, 48) -0.026
(34, 49) 0.005
(34, 50) 0.025
(34, 51) -0.042
(34, 52) -0.013
(34, 53) 0.028
(34, 54) -0.011
(34, 55) 0.019
(34, 56) -0.003
(34, 57) -0.034
(34, 58) 0.027
(34, 59) 0.128
(34, 60) -0.092
(34, 61) -0.056
(34, 62) 0

(42, 86) -0.008
(42, 87) -0.035
(42, 88) -0.043
(42, 89) 0.026
(42, 90) -0.01
(42, 91) -0.041
(42, 92) 0.007
(42, 93) 0.021
(42, 94) 0.008
(42, 95) 0.022
(42, 96) 0.001
(42, 97) 0.007
(42, 98) 0.025
(42, 99) 0.007
(43, 44) -0.002
(43, 45) 0.024
(43, 46) 0.003
(43, 47) 0.076
(43, 48) 0.002
(43, 49) 0.012
(43, 50) 0.022
(43, 51) -0.004
(43, 52) 0.013
(43, 53) -0.038
(43, 54) -0.003
(43, 55) -0.004
(43, 56) 0.001
(43, 57) -0.004
(43, 58) -0.005
(43, 59) 0.093
(43, 60) 0.015
(43, 61) 0.012
(43, 62) 0.001
(43, 63) -0.041
(43, 64) -0.003
(43, 65) -0.051
(43, 66) -0.002
(43, 67) -0.046
(43, 68) -0.038
(43, 69) 0.002
(43, 70) 0.086
(43, 71) -0.049
(43, 72) 0.028
(43, 73) 0.021
(43, 74) 0.013
(43, 75) 0.003
(43, 76) -0.006
(43, 77) -0.002
(43, 78) 0.027
(43, 79) 0.014
(43, 80) -0.017
(43, 81) 0.023
(43, 82) -0.01
(43, 83) 0.002
(43, 84) 0.014
(43, 85) 0.01
(43, 86) 0.013
(43, 87) -0.001
(43, 88) -0.038
(43, 89) -0.011
(43, 90) -0.046
(43, 91) -0.003
(43, 92) 0.008
(43, 93) 0.025
(43, 94) -0.006

(53, 73) 0.008
(53, 74) -0.066
(53, 75) 0.015
(53, 76) 0.017
(53, 77) -0.001
(53, 78) -0.012
(53, 79) -0.016
(53, 80) -0.02
(53, 81) -0.047
(53, 82) -0.036
(53, 83) 0.011
(53, 84) 0.043
(53, 85) -0.044
(53, 86) -0.017
(53, 87) -0.034
(53, 88) -0.002
(53, 89) 0.019
(53, 90) 0.053
(53, 91) -0.012
(53, 92) 0.014
(53, 93) -0.016
(53, 94) 0.011
(53, 95) -0.042
(53, 96) -0.006
(53, 97) -0.028
(53, 98) -0.029
(53, 99) 0.017
(54, 55) 0.002
(54, 56) 0.006
(54, 57) -0.008
(54, 58) 0.002
(54, 59) -0.005
(54, 60) 0.008
(54, 61) -0.02
(54, 62) 0.001
(54, 63) -0.035
(54, 64) -0.013
(54, 65) -0.007
(54, 66) -0.004
(54, 67) 0.002
(54, 68) -0.003
(54, 69) 0.001
(54, 70) -0.002
(54, 71) -0.016
(54, 72) -0.023
(54, 73) 0.001
(54, 74) -0.01
(54, 75) -0.0
(54, 76) -0.004
(54, 77) -0.0
(54, 78) -0.013
(54, 79) 0.014
(54, 80) -0.002
(54, 81) -0.013
(54, 82) 0.01
(54, 83) 0.006
(54, 84) 0.001
(54, 85) 0.009
(54, 86) -0.004
(54, 87) 0.003
(54, 88) 0.02
(54, 89) -0.009
(54, 90) 0.002
(54, 91) -0.011
(54, 92) 0.

(67, 74) 0.003
(67, 75) -0.006
(67, 76) 0.009
(67, 77) -0.001
(67, 78) 0.014
(67, 79) -0.0
(67, 80) 0.01
(67, 81) -0.034
(67, 82) -0.01
(67, 83) -0.004
(67, 84) 0.023
(67, 85) -0.052
(67, 86) -0.0
(67, 87) -0.018
(67, 88) 0.037
(67, 89) 0.007
(67, 90) -0.012
(67, 91) 0.02
(67, 92) 0.003
(67, 93) -0.008
(67, 94) 0.008
(67, 95) -0.014
(67, 96) 0.001
(67, 97) -0.0
(67, 98) 0.003
(67, 99) 0.014
(68, 69) 0.011
(68, 70) 0.011
(68, 71) -0.028
(68, 72) 0.01
(68, 73) -0.015
(68, 74) -0.009
(68, 75) -0.006
(68, 76) 0.025
(68, 77) -0.002
(68, 78) 0.011
(68, 79) 0.014
(68, 80) 0.011
(68, 81) 0.048
(68, 82) 0.018
(68, 83) -0.003
(68, 84) -0.014
(68, 85) -0.016
(68, 86) 0.011
(68, 87) -0.024
(68, 88) -0.005
(68, 89) 0.002
(68, 90) 0.026
(68, 91) 0.008
(68, 92) -0.031
(68, 93) -0.0
(68, 94) 0.031
(68, 95) -0.013
(68, 96) 0.006
(68, 97) -0.04
(68, 98) -0.019
(68, 99) -0.001
(69, 70) 0.004
(69, 71) 0.015
(69, 72) 0.002
(69, 73) 0.002
(69, 74) 0.012
(69, 75) -0.0
(69, 76) -0.014
(69, 77) 0.001
(69, 78) 

In [11]:
len(skim.selected_covariates())

100

In [8]:
X_valid

DeviceArray([[ 0.17453367, -1.1035134 , -0.6988685 , ...,  0.22002265,
              -0.34613922, -1.4543749 ],
             [ 1.87481   , -0.5956119 ,  0.845807  , ...,  2.2265294 ,
               0.55536383,  1.0615324 ],
             [ 0.25740656,  1.1475091 ,  0.01667643, ...,  0.03704556,
              -0.03753162,  0.97074306],
             ...,
             [-1.7974405 , -1.4911877 ,  1.0353976 , ...,  0.8979323 ,
               0.90127057,  0.9054762 ],
             [ 0.964937  , -0.3890809 , -0.8998068 , ...,  0.589313  ,
              -0.32199076,  0.1622231 ],
             [ 1.125396  , -1.6570342 , -0.6687731 , ..., -0.27552766,
               0.10153985, -0.22743613]], dtype=float32)

In [10]:
lanova.get_variation_at_order(featprocessor.transform(X_valid), 2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 4950/4950 [00:28<00:00, 174.73it/s]


DeviceArray([-1.4430153e+00, -9.2751664e-01,  9.8711479e-01,
             -1.7791083e+00,  6.5272517e+00,  2.2221732e-01,
              7.8650141e-01,  1.9590884e-01, -5.5259019e-01,
              5.3683457e+00,  3.5494483e-01,  1.7244517e+00,
             -1.7263447e+00,  5.7682605e+00,  5.0312752e-01,
             -7.5604808e-01,  3.6134977e+00,  1.9442846e+00,
              5.4958028e-01,  6.1505592e-01,  3.0771792e+00,
              1.4533045e+00,  8.2404107e-01,  1.1950012e+00,
             -6.9547836e-03, -7.3267275e-01,  1.1949853e+00,
             -1.0750980e+00,  1.9716014e+00,  2.7599487e+00,
             -1.9900014e+00,  2.0536373e+00, -3.0185816e-01,
             -3.4264094e-01, -3.8851786e-01,  5.7688171e-01,
             -2.9801705e+00, -1.5904459e+00, -3.4980166e-01,
             -1.4547993e+00, -3.8563790e+00, -8.1280434e-01,
             -3.0574839e+00,  3.0672925e+00,  3.8054719e+00,
              2.3415594e-01,  7.4335271e-01, -6.9976360e-01,
             -2.0261061e